This notebook contains exercises related to Capstone Project 1.

Instacart data structure:
   - orders: list of all orders in the dataset. 1 row per order, includes day of the week and time of the day;
   - order_products train: which products (product_id) were ordered; in which order products were added to the cart, and if the product is re-order (1) or not (0);
   - contain the names of products with their corresponding product_id;
   - order_products prior: the structure is the same as order_product_train.csv;
   - aisles contains the aisles;
   - departments: contains the department names.

In [8]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving instacart_prior.zip to instacart_prior.zip
User uploaded file "instacart_prior.zip" with length 164686327 bytes


In [9]:
!unzip instacart_prior.zip
!unzip instacart_orders.zip
!ls

Archive:  instacart_prior.zip
  inflating: order_products_prior.csv  
Archive:  instacart_orders.zip
  inflating: orders.csv              
aisles.csv	      instacart_prior.zip	orders.csv
departments.csv       order_products_prior.csv	products.csv
instacart_orders.zip  order_products_train.csv	sample_data


In [0]:
# We import the needed packages.
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
import csv
import seaborn as sns
sns.set()
pd.options.display.latex.repr=True

In [0]:
# Execute if needed.
# Strings to replace names of files run in online environment.
str_isles = 'aisles.csv'
str_dept = 'departments.csv'
str_prod = 'products.csv'
str_prod_train = 'order_products_train.csv'
str_prod_prior = 'order_products_prior.csv'
str_orders = 'orders.csv'

In [0]:
# Execute if needed.
# Strings for local machine runs
str_isles = 'data/aisles.csv'
str_dept = 'data/departments.csv'
str_prod = 'data/products.csv'
str_prod_train = 'data/order_products_train.csv'
str_prod_prior = 'data/order_products_prior.csv'
str_orders = 'data/orders.csv'

We read the files for data and check them.

For the isles, department frames, we change these into categories.

In [12]:
# isles frame
isles = pd.read_csv(str_isles, index_col='aisle_id')
isles['aisle'].astype('category')
isles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134 entries, 1 to 134
Data columns (total 1 columns):
aisle    134 non-null object
dtypes: object(1)
memory usage: 2.1+ KB


In [13]:
isles.head()

,aisle
aisle_id,
1,prepared soups salads
2,specialty cheeses
3,energy granola bars
4,instant foods
5,marinades meat preparation


We change the type of the department data to category.

In [14]:
# department frame dept
dept = pd.read_csv(str_dept, index_col='department_id')
dept.department.astype('category')
dept.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 1 to 21
Data columns (total 1 columns):
department    21 non-null object
dtypes: object(1)
memory usage: 336.0+ bytes


In [15]:
dept.head()

,department
department_id,
1,frozen
2,other
3,bakery
4,produce
5,alcohol


In [16]:
# product frame
prod = pd.read_csv(str_prod, index_col='product_id')
prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 1 to 49688
Data columns (total 3 columns):
product_name     49688 non-null object
aisle_id         49688 non-null int64
department_id    49688 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [17]:
prod.head()

,product_name,aisle_id,department_id
product_id,,,
1,Chocolate Sandwich Cookies,61,19
2,All-Seasons Salt,104,13
3,Robust Golden Unsweetened Oolong Tea,94,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
5,Green Chile Anytime Sauce,5,13


In [18]:
# train frame
train = pd.read_csv(str_prod_train, index_col='order_id')
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384617 entries, 1 to 3421070
Data columns (total 3 columns):
product_id           1384617 non-null int64
add_to_cart_order    1384617 non-null int64
reordered            1384617 non-null int64
dtypes: int64(3)
memory usage: 42.3 MB


/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [19]:
# prior frame
prior = pd.read_csv(str_prod_prior,  index_col = 'order_id')
prior.info()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 2 to 3421083
Data columns (total 3 columns):
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(3)
memory usage: 989.8 MB


We change eval_set to category. The column has only three values.

In [20]:
orders = pd.read_csv(str_orders, index_col= 'order_id')
orders.eval_set.astype('category')
orders.head(10)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,
2539329,1,prior,1,2,8,NaN
2398795,1,prior,2,3,7,15.0
473747,1,prior,3,3,12,21.0
2254736,1,prior,4,4,7,29.0
431534,1,prior,5,4,15,28.0
3367565,1,prior,6,2,7,19.0
550135,1,prior,7,1,9,20.0
3108588,1,prior,8,1,14,14.0
2295261,1,prior,9,1,16,0.0


In [21]:
orders.eval_set.unique()

array(['prior', 'train', 'test'], dtype=object)

Combining train and prior data frames.

In [23]:
# train and prior contain the same type of data. We combine these two for analysis.
combine = pd.concat([prior, train])
combine.head()

,product_id,add_to_cart_order,reordered
order_id,,,
2,33120,1,1
2,28985,2,1
2,9327,3,0
2,45918,4,1
2,30035,5,0


We visualize the data.

We check to see data in hourly categories.

 We visualize data for inspection.
 
 We see that all orders for 30 days or more are aggregated into a single column.

We cannot work on pattern where the days are 30 or more, because all orders are aggregated. We are separating these orders from set.

We save orders in two separate files for further processing: orders_29.csv and orders_30.csv. 

The first file contains all data for orders that are reordes 29 days or less after a prior order. 

The other file contains all orders that are 30 days or more after the previous order.

We investigate relationship between days sinse prior order and number of items purchased. We use trunkated file, because 30 days is an agregate that contains 30 or more days.

In [0]:
last_item = combine.groupby('order_id').count()['add_to_cart_order']


In [0]:
df_all = pd.merge(left=orders, right=combine, left_on='order_id', right_on='order_id')

In [25]:
df_all.head()

,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
order_id,,,,,,,,,
2539329,1,prior,1,2,8,NaN,196,1,0
2539329,1,prior,1,2,8,NaN,14084,2,0
2539329,1,prior,1,2,8,NaN,12427,3,0
2539329,1,prior,1,2,8,NaN,26088,4,0
2539329,1,prior,1,2,8,NaN,26405,5,0


In [26]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 2539329 to 272231
Data columns (total 9 columns):
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
product_id                int64
add_to_cart_order         int64
reordered                 int64
dtypes: float64(1), int64(7), object(1)
memory usage: 2.5+ GB


In [27]:
prod_all = pd.merge(left=prod, right=dept, left_on='department_id', right_on='department_id')
prod_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 0 to 49687
Data columns (total 4 columns):
product_name     49688 non-null object
aisle_id         49688 non-null int64
department_id    49688 non-null int64
department       49688 non-null object
dtypes: int64(2), object(2)
memory usage: 1.9+ MB


In [28]:
prod_all = pd.merge(left=prod_all, right=isles, left_on='aisle_id', right_on='aisle_id')
prod_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 0 to 49687
Data columns (total 5 columns):
product_name     49688 non-null object
aisle_id         49688 non-null int64
department_id    49688 non-null int64
department       49688 non-null object
aisle            49688 non-null object
dtypes: int64(2), object(3)
memory usage: 2.3+ MB


In [29]:
prod_all=prod_all[['product_name', 'aisle', 'department']]
prod_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 0 to 49687
Data columns (total 3 columns):
product_name    49688 non-null object
aisle           49688 non-null object
department      49688 non-null object
dtypes: object(3)
memory usage: 1.5+ MB


In [30]:
prod_all.head()

,product_name,aisle,department
0,Chocolate Sandwich Cookies,cookies cakes,snacks
1,Nutter Butter Cookie Bites Go-Pak,cookies cakes,snacks
2,Danish Butter Cookies,cookies cakes,snacks
3,Gluten Free All Natural Chocolate Chip Cookies,cookies cakes,snacks
4,Mini Nilla Wafers Munch Pack,cookies cakes,snacks


In [31]:
df_all = pd.merge(left=df_all, right=prod_all, left_on='product_id', right_index=True)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819013 entries, 2539329 to 3093936
Data columns (total 12 columns):
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
product_id                int64
add_to_cart_order         int64
reordered                 int64
product_name              object
aisle                     object
department                object
dtypes: float64(1), int64(7), object(4)
memory usage: 3.3+ GB


In [32]:
df_all.head()

,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle,department
order_id,,,,,,,,,,,,
2539329,1,prior,1,2,8,NaN,196,1,0,Simply Made Chocolate Chip Cookies,cookies cakes,snacks
2398795,1,prior,2,3,7,15.0,196,1,1,Simply Made Chocolate Chip Cookies,cookies cakes,snacks
473747,1,prior,3,3,12,21.0,196,1,1,Simply Made Chocolate Chip Cookies,cookies cakes,snacks
2254736,1,prior,4,4,7,29.0,196,1,1,Simply Made Chocolate Chip Cookies,cookies cakes,snacks
431534,1,prior,5,4,15,28.0,196,1,1,Simply Made Chocolate Chip Cookies,cookies cakes,snacks


In [0]:
del df_all['product_id']

In [34]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819013 entries, 2539329 to 3093936
Data columns (total 11 columns):
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
add_to_cart_order         int64
reordered                 int64
product_name              object
aisle                     object
department                object
dtypes: float64(1), int64(6), object(4)
memory usage: 3.0+ GB


In [0]:
# saving the ready dataframe for further cleaning and exploration

df_all.to_csv('combined_data.csv', chunksize=1000000)

In [0]:
df_all = pd.read_csv('combined_data.csv')

In [38]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33819013 entries, 0 to 33819012
Data columns (total 12 columns):
order_id                  int64
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
add_to_cart_order         int64
reordered                 int64
product_name              object
aisle                     object
department                object
dtypes: float64(1), int64(7), object(4)
memory usage: 3.0+ GB


In [39]:
!zip combined_data.zip combined_data.csv

  adding: combined_data.csv (deflated 86%)


In [0]:
name_base = 'combined_data'
name_end = ['00.csv', '01.csv', '02.csv', '03.csv', '04.csv', '05.csv', '06.csv', '07.csv', '08.csv', '09.csv', 
           '10.csv', '11.csv', '12.csv', '13.csv', '14.csv', '15.csv', '16.csv']


file_name = []
for end in name_end:
  file_name.append(name_base + str(end))


In [0]:
#function split large dataframe into chunks
def split_frame(df, chunksize=10000):
  list_of_frames = list()
  num_chunks = len(df) // chunksize +1
  for i in range(num_chunks):
    list_of_frames.append(df[i*chunksize: (i+1)*chunksize])
  return list_of_frames

In [0]:
small_frames = split_frame(df_all, 2000000)
small_frames

In [0]:
small_frames[16].to_csv(file_name[16])


In [61]:
! zip combined_data00.zip combined_data??.csv

  adding: combined_data00.csv (deflated 85%)
  adding: combined_data01.csv (deflated 85%)
  adding: combined_data02.csv (deflated 85%)
  adding: combined_data03.csv (deflated 85%)
  adding: combined_data04.csv (deflated 84%)
  adding: combined_data05.csv (deflated 85%)
  adding: combined_data06.csv (deflated 85%)
  adding: combined_data07.csv (deflated 85%)
  adding: combined_data08.csv (deflated 85%)
  adding: combined_data09.csv (deflated 85%)
  adding: combined_data10.csv (deflated 85%)
  adding: combined_data11.csv (deflated 85%)
  adding: combined_data12.csv (deflated 85%)
  adding: combined_data13.csv (deflated 85%)
  adding: combined_data14.csv (deflated 84%)
  adding: combined_data15.csv (deflated 84%)
  adding: combined_data16.csv (deflated 83%)
